<a href="https://colab.research.google.com/github/vaanchhitbaranwal-ux/vaanchhit/blob/main/Social_Media_Marketing_Analysis_MMM_PYMC_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv('conjura_mmm_data.csv')
df.head()

In [ ]:
df.isnull().mean()

In [ ]:
df.dtypes

In [ ]:
df['DATE_DAY'] = pd.to_datetime(df['DATE_DAY'])

In [ ]:
# numerical features
numerical_features = df.select_dtypes(include = ['int64', 'float64']).columns.to_list()

# categorical features
categorical_features = df.select_dtypes(include = ['object']).columns.to_list()

In [ ]:
# finding number of unique elements in each categorical variable
df[categorical_features].nunique(axis=0)

In [ ]:
uniq_ele = df[categorical_features].iloc[:,2:].columns
for x in uniq_ele:
    sns.countplot(x = x, data = df)
    plt.xticks(rotation=90)
    plt.show(

In [ ]:
df['ORGANISATION_ID'].value_counts().nlargest(5)

In [ ]:
df['DATE_DAY'].max(), df['DATE_DAY'].min()

In [ ]:
df1 = df.loc[df['ORGANISATION_ID'] == 'ba773ebd7ec0a08f1d042187d086ccb4']

In [ ]:
# understanding unique elements in the categorical data
uniq_ele = df1[categorical_features].iloc[:,2:].columns
for x in uniq_ele:
    sns.countplot(x = x, data = df1)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
df1['revenue'] = (df1['ALL_PURCHASES_ORIGINAL_PRICE']*df1['ALL_PURCHASES_UNITS'])-df1['FIRST_PURCHASES_GROSS_DISCOUNT']

In [ ]:
df1['MMM_TIMESERIES_ID'].value_counts()

In [ ]:
df1.loc[df1['TERRITORY_NAME']== 'Germany']['MMM_TIMESERIES_ID'].unique()

In [ ]:
plt.figure(figsize= (40,10))
sns.lineplot(x = df1['DATE_DAY'], y = df1['ALL_PURCHASES_UNITS'], data = df1, hue = df1['TERRITORY_NAME'])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# it is noticeable that All Territories have the aggregated data of purchased units of all countries.
# to avoid currency conversion rate dispute, we will be using All Territories since it uses the currency
# organization's primary territory i.e. GB
df2 = df1.loc[df1['TERRITORY_NAME'] == 'All Territories']

In [ ]:
for x in df2[categorical_features].columns:
    print(x, df2[x].unique())

In [ ]:
clicks_impressions = ['GOOGLE_PAID_SEARCH_CLICKS',
 'GOOGLE_SHOPPING_CLICKS',
 'GOOGLE_PMAX_CLICKS',
 'GOOGLE_DISPLAY_CLICKS',
 'GOOGLE_VIDEO_CLICKS',
 'META_FACEBOOK_CLICKS',
 'META_INSTAGRAM_CLICKS',
 'META_OTHER_CLICKS',
 'TIKTOK_CLICKS',
 'GOOGLE_PAID_SEARCH_IMPRESSIONS',
 'GOOGLE_SHOPPING_IMPRESSIONS',
 'GOOGLE_PMAX_IMPRESSIONS',
 'GOOGLE_DISPLAY_IMPRESSIONS',
 'GOOGLE_VIDEO_IMPRESSIONS',
 'META_FACEBOOK_IMPRESSIONS',
 'META_INSTAGRAM_IMPRESSIONS',
 'META_OTHER_IMPRESSIONS',
 'TIKTOK_IMPRESSIONS',
 'DIRECT_CLICKS',
 'BRANDED_SEARCH_CLICKS',
 'ORGANIC_SEARCH_CLICKS',
 'EMAIL_CLICKS',
 'REFERRAL_CLICKS',
 'ALL_OTHER_CLICKS']

In [ ]:
# filling nans with 0 because these nans might be an indication that investments in specific
# fields were not done due to different factors like non-acquisition season, unknown sector for
# the company
df2.fillna(0, inplace=True)

In [ ]:
plt.figure(figsize=(40, 20))

# plotting the lines for clicks_impressions
for x in clicks_impressions:
    sns.lineplot(x='DATE_DAY', y=x, data=df2, label=x)

# creating a secondary y-axis for 'revenue'
ax2 = plt.gca().twinx()
sns.lineplot(x='DATE_DAY', y='revenue', data=df2, ax=ax2, color='r', label='revenue')

# setting labels and legend
plt.ylabel('Clicks/Impressions')
ax2.set_ylabel('Revenue')
plt.xticks(rotation=90)
plt.legend(loc='upper left', bbox_to_anchor=(0, 1))

plt.show()

In [ ]:
plt.figure(figsize = (20,10))
cor = df2.iloc[:, 10:].corr()
cor = cor[((cor < -0.5) & (cor > -1)) | ((cor > 0.5) & (cor < 1))]
sns.heatmap(cor, annot = True, cmap = 'icefire')

In [ ]:
def find_correlation(df, thresh):

    corrMatrix = df.corr()
    corrMatrix.loc[:,:] =  np.tril(corrMatrix, k=-1)

    already_in = set()
    result = []

    for col in corrMatrix:
        perfect_corr = corrMatrix[col][corrMatrix[col] > thresh].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)


    select_nested = [f[1:] for f in result]
    select_flat = [i for j in select_nested for i in j]
    return select_flat

find_correlation(df2.iloc[:,10:], 0.8)

In [ ]:
df2.drop(['MMM_TIMESERIES_ID', 'ORGANISATION_ID', 'ORGANISATION_VERTICAL',
           'ORGANISATION_SUBVERTICAL', 'ORGANISATION_MARKETING_SOURCES',
           'ORGANISATION_PRIMARY_TERRITORY_NAME', 'TERRITORY_NAME', 'CURRENCY_CODE',
         'ALL_PURCHASES'],
         inplace= True, axis = 1)

In [ ]:
import statsmodels.api as sm
results = sm.OLS(df2['ALL_PURCHASES_UNITS'], df2.iloc[:,1:].drop('ALL_PURCHASES_UNITS',
                                                                 axis = 1)).fit()

results.summary()

In [ ]:
df2.drop(['GOOGLE_PAID_SEARCH_CLICKS', 'GOOGLE_DISPLAY_CLICKS', 'GOOGLE_VIDEO_CLICKS',
          'META_FACEBOOK_CLICKS', 'META_INSTAGRAM_CLICKS', 'TIKTOK_CLICKS',
          'GOOGLE_DISPLAY_IMPRESSIONS', 'GOOGLE_VIDEO_IMPRESSIONS', 'META_FACEBOOK_IMPRESSIONS',
          'META_OTHER_IMPRESSIONS', 'TIKTOK_IMPRESSIONS', 'DIRECT_CLICKS', 'REFERRAL_CLICKS'],
         inplace = True, axis = 1)

In [ ]:
spend_vars = [
    'GOOGLE_PAID_SEARCH_SPEND',
    'GOOGLE_SHOPPING_SPEND',
    'GOOGLE_PMAX_SPEND',
    'META_FACEBOOK_SPEND',
    'META_INSTAGRAM_SPEND'
]

clicks_impressions_vars = [
                'FIRST_PURCHASES_UNITS',
                'FIRST_PURCHASES_ORIGINAL_PRICE',
                'FIRST_PURCHASES_GROSS_DISCOUNT',
                'ALL_PURCHASES_ORIGINAL_PRICE',
                'ALL_PURCHASES_GROSS_DISCOUNT',
                'GOOGLE_SHOPPING_CLICKS',
                'GOOGLE_PMAX_CLICKS',
                'META_OTHER_CLICKS',
                'GOOGLE_PAID_SEARCH_IMPRESSIONS',
                'GOOGLE_SHOPPING_IMPRESSIONS',
                'GOOGLE_PMAX_IMPRESSIONS',
                'META_INSTAGRAM_IMPRESSIONS',
                'BRANDED_SEARCH_CLICKS',
                'ORGANIC_SEARCH_CLICKS',
                'EMAIL_CLICKS',
                'ALL_OTHER_CLICKS'
               ]

date_var = 'DATE_DAY'

kpi = 'ALL_PURCHASES_UNITS'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df2.set_index(date_var, inplace=True)
df2 = df2.resample('W').sum()

In [ ]:
# convert 'DATE_DAY' to datetime and extract year
df2['year'] = df2.index.to_series().dt.year

In [ ]:
# group by year and sum each category
spend_data = df2.groupby('year')[spend_vars].sum()
clicks_data = df2.groupby('year')[clicks_impressions_vars].sum(

In [ ]:
# plotting the spend variables
plt.figure(figsize=(10, 6))
spend_data.plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('Annual Spend Data Grouped by Year')
plt.xlabel('Year')
plt.ylabel('Spend Amount')
plt.legend(title='Spend Categories', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10, 6))
clicks_data.plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('Annual Control Variables Data Grouped by Year')
plt.xlabel('Year')
plt.ylabel('Control Variable Metrics')
plt.legend(title='Control Categories', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

plt.show()

In [ ]:
sns.lineplot(x=df2.index, y = df2['ALL_PURCHASES_UNITS'])
plt.xticks(rotation = 90)
ax2 = plt.gca().twinx()
sns.lineplot(x=df2.index, y = df2['ALL_PURCHASES_GROSS_DISCOUNT'], ax=ax2, color='r', label='discounts', alpha = 0.5)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
control_vars = ['ALL_PURCHASES_GROSS_DISCOUNT']

In [ ]:
df2.drop(['GOOGLE_VIDEO_SPEND', 'TIKTOK_SPEND', 'META_OTHER_SPEND'], axis =1 , inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test, y_train, y_test = train_test_split(df2.drop('ALL_PURCHASES_UNITS', axis=1),
                                                      df2['ALL_PURCHASES_UNITS'],
                                                      test_size= 0.2,
                                                      shuffle=False)

In [ ]:
spend_by_channel = df2[spend_vars].sum(axis=0)
spend_share = spend_by_channel / spend_by_channel.sum()

HALFNORMAL_SCALE = 1 / np.sqrt(1 - 2 / np.pi)

n_channels = len(spend_vars)

prior_sigma = HALFNORMAL_SCALE * n_channels * spend_share.to_numpy()
prior_sigma.tolist()

In [ ]:
mmm_config = {
    'intercept': {'dist': 'LogNormal',
              'kwargs': {'mu': 0.5, 'sigma': 2}},

    'beta_channel': {'dist': 'LogNormal', # parameters for our media channels
                     'kwargs': {'mu':1, 'sigma': prior_sigma}},

    'alpha': {'dist': 'Beta', # parameter for adstock function for medial channels
              'kwargs': {'alpha': 1, 'beta': 3}},

    'lam': {'dist': 'Gamma',  # parameter for the saturation function for media channels
            'kwargs': {'alpha': 3, 'beta': 1}},

    'likelihood': {'dist': 'Normal',
                   'kwargs': {'sigma': {'dist': 'HalfNormal', 'kwargs': {'sigma': 2}}}},

    'gamma_control': {'dist': 'Normal', # parameter for any control variables
                      'kwargs': {'mu': 0, 'sigma': 1}},

    'gamma_fourier': {'dist': 'Laplace', # parameter for fourier series to model seasonality
                      'kwargs': {'mu': 0, 'b': 3}}
            }

In [ ]:
from pymc_marketing.mmm.transformers import geometric_adstock, logistic_saturation
from pymc_marketing.mmm.delayed_saturated_mmm import DelayedSaturatedMMM

In [ ]:
# progress bar
sampler_config= {"progressbar": True}

In [ ]:
#building the model
mmm = DelayedSaturatedMMM(
    model_config = mmm_config,
    sampler_config = sampler_config,
    date_column = date_var,
    channel_columns = spend_vars,
    control_columns = control_vars,
    adstock_max_lag=8,
    yearly_seasonality=2,
)

In [ ]:
# fitting the model
mmm.fit(X=df_train.reset_index(), y=y_train, target_accept=0.90, chains=4,
        random_seed=123)

In [ ]:
mmm.fit_result

In [ ]:
az.summary(
    data=mmm.fit_result,
    var_names=[
        'intercept',
        'beta_channel',
        'alpha',
        'lam',
        'gamma_fourier',
        'likelihood_sigma'
    ],
)

In [ ]:
_ = az.plot_trace(
    data=mmm.fit_result,
    var_names=[
        'intercept',
        'beta_channel',
        'alpha',
        'lam',
        'gamma_fourier',
        'likelihood_sigma'
    ],
    compact=True,
    backend_kwargs={"figsize": (12, 10), "layout": "constrained"},
)
plt.gcf().suptitle("Model Trace", fontsize=16);

In [ ]:
y_train_pred = mmm.sample_posterior_predictive(df_train.reset_index(), extend_idata=True, combined=True)
mmm.plot_posterior_predictive(original_scale=True)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

y_train_pred_mean = y_train_pred['y'].mean(dim='sample').values

r_squared_training = r2_score(y_train, y_train_pred_mean)

mape = mean_absolute_percentage_error(y_train, y_train_pred_mean)

print('R-Squared score: ', r_squared_training)
print('Mean Absolute Percentage Error: ', mape)

In [ ]:
# channel contributions to revenue

fig = mmm.plot_channel_contribution_share_hdi(figsize=(7, 5))

In [ ]:
# channel contribution

df_attribution = (
    mmm.compute_channel_contribution_original_scale()
    .mean(dim=('chain','draw'))
    .to_dataframe(name='attribution')
    .reset_index()
    .pivot(index='date', columns='channel', values='attribution')
    .reset_index()
    .rename_axis(None, axis=1)
)

column_order = df_attribution.drop('date', axis=1).sum(axis=0).sort_values(ascending=False).index

fig, ax = plt.subplots(figsize=(10, 6))

bottom = 0  # Initialize bottom value for stacking

for channel in column_order:
    ax.fill_between(df_attribution['date'], bottom, bottom + df_attribution[channel], label=channel, alpha=0.7)
    bottom += df_attribution[channel]

plt.xlabel('Date')
plt.ylabel('Revenue Contribution')
plt.title('Channel Contributions Over Time')
plt.legend(title='Channel', loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
mmm.plot_components_contributions()
plt.xticks(rotation = 90)
plt.show()

In [ ]:
mmm.plot_channel_parameter(param_name="alpha", figsize=(9, 5))
plt.show()

In [ ]:
# Calculate ROAS samples using optimized code
channel_contribution_original_scale = mmm.compute_channel_contribution_original_scale()

# Sum over date dimension and calculate ROAS
roas_samples = (
    channel_contribution_original_scale.stack(sample=("chain", "draw")).sum("date")
    / df_train[spend_vars].sum().to_numpy()[..., None]
)

# Create histogram plot
fig, ax = plt.subplots(figsize=(10, 6))

for channel in spend_vars:
    data = roas_samples.sel(channel=channel).to_numpy()

    # Filter out NaN values
    data = data[~np.isnan(data)]

    # Plot histogram
    sns.histplot(data, label=channel, binwidth=0.05, alpha=0.3, kde=True, ax=ax)

    # Calculate and display mean for each channel
    median_value = np.median(data)
    ax.axvline(median_value, linestyle='dashed', color=sns.color_palette()[spend_vars.index(channel)], alpha=0.7, linewidth=1)

ax.set_xlim(0, 4)
ax.set_ylim(0, 4500)  # Set y-axis limit to 5000
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))
ax.set(title="Posterior ROAS distribution", xlabel="ROAS", ylabel="Frequency")

plt.tight_layout()
plt.show()

In [ ]:
#let's get one number for ROAS for each channel
roas_samples.median(dim='sample').to_dataframe(name='median_ROAS').sort_values(by='median_ROAS',ascending=False)

In [ ]:
# Generate predictions on the test set

y_test_pred = mmm.sample_posterior_predictive(X_pred=df_test.reset_index(), extend_idata=True,
                                              include_last_observations=True)
y_test_pred

In [ ]:
def plot_in_sample(X, y, ax, n_points: int=0):
    (
        y.to_frame()
        .set_index(X[date_var])
        .iloc[-n_points:]
        .plot(ax=ax, color="black", label="actuals")
    )

def plot_out_of_sample(X_out_of_sample, y_out_of_sample, ax, color, label):
    y_out_of_sample_groupby = y_out_of_sample["y"].to_series().groupby('date')

    lower, upper = quantiles = [0.025, 0.975]
    conf = y_out_of_sample_groupby.quantile(quantiles).unstack()
    ax.fill_between(
        X_out_of_sample[date_var].dt.to_pydatetime(),
        conf[lower],
        conf[upper],
        alpha=0.25,
        color=color,
        label=f"{label} 95% HDI",
    )

    mean = y_out_of_sample_groupby.mean()
    mean.plot(ax=ax, label=label, color=color, linestyle="--")
    ax.set(
        ylabel="Revenue",
        title="Out of sample predictions"
    )

    return ax

fig, ax = plt.subplots(figsize=(12,7))
plot_in_sample(df2.drop(kpi, axis=1).reset_index(), df2[kpi], n_points=365, ax=ax)
plot_out_of_sample(df_test.reset_index(), y_test_pred, ax=ax, label="out of sample", color="C0")
ax.legend();
plt.show()

In [ ]:
#Check the r2 using the mean of the y_test_pred posterior

y_test_pred_mean = y_test_pred['y'].mean(dim='sample').values

mape = mean_absolute_percentage_error(y_test, y_test_pred_mean)

print('Mean Absolute Percentage Error: ', mape)

In [ ]:
total_budget = 3 # we are assuming that 3M GBP has been alloted to the marketing team

# initial split per channel
budget_per_channel = total_budget / n_channels

# initial budget per channel as dictionary
initial_budget_dict = {channel: budget_per_channel for channel in spend_vars}

# bounds for each channel - minimum 1k, maximum 750k
min_budget, max_budget = .001, 0.75
budget_bounds = {channel: [min_budget, max_budget] for channel in spend_vars}

In [ ]:
mmm.plot_direct_contribution_curves(show_fit=True)

In [ ]:
# calculating parameters for sigmoid function
# sigmoid function is being considered becaause we
# expect a process to have a slow start, rapid growth in the middle,
# and then a slow approach to an upper asymptote.
sigmoid_params = mmm.compute_channel_curve_optimization_parameters_original_scale(method='sigmoid')

In [ ]:
result_sigmoid = mmm.optimize_channel_budget_for_maximum_contribution(
    method = 'sigmoid', #define saturation function
    total_budget = total_budget,
    parameters = sigmoid_params,
    budget_bounds = budget_bounds
)

result_sigmoid

In [ ]:
df2[spend_vars].groupby(df2['year']==2023).sum()

In [ ]:
budget_bounds['GOOGLE_PAID_SEARCH_SPEND'] = [.5, .8]
budget_bounds['GOOGLE_SHOPPING_SPEND'] = [0.3, 0.5]
budget_bounds['GOOGLE_PMAX_SPEND'] = [.25, .5]
budget_bounds['META_FACEBOOK_SPEND'] = [.02, .03]
budget_bounds['META_INSTAGRAM_SPEND'] = [.01, 0.05]

budget_bounds

In [ ]:
result_sigmoid = mmm.optimize_channel_budget_for_maximum_contribution(
    method = 'sigmoid', #define saturation function
    total_budget = total_budget,
    parameters = sigmoid_params,
    budget_bounds = budget_bounds
)

result_sigmoid.sort_values(by='estimated_contribution', ascending = False)

In [ ]:
# use the function `calculate_expected_contribution` to estimate
# the contribution of your initial budget based on the curve parameters.
from pymc_marketing.mmm.budget_optimizer import calculate_expected_contribution

initial_contribution = calculate_expected_contribution(
    method='sigmoid',
    parameters = sigmoid_params,
    budget = initial_budget_dict
)

# initial budget & contribution dictionary
initial_scenario = {
    'initial_contribution': initial_contribution,
    'initial_budget': initial_budget_dict
}

In [ ]:
figure_ = mmm.plot_budget_scenearios(base_data=initial_scenario, method='sigmoid', scenarios_data=[result_sigmoid])